## Homework 12

Finish the development of your code to construct persistence diagrams.

Use your code to examine some data and find persistent homologies, and do some simple analysis.

Your code must do all the following:

1. Read in the data.  CSV files (M_data<indexnumber>.csv) have been uploaded to the blackboard site. These files are comma separated value files, have no text header row, so you need to know each row starts with an identifier number and then has the 10-dimensional feature vector of the data. Each file has something like 75 items.

2. Use a Euclidean metric (without any scaling or taking out correlations).

3. Rank order the metric distances from point to point, so you can filter on thresholds from below the minimum space between any two points to above the maximum space between any two points.

4. For each value of the threshold (it your code runs fast enough step the threshold between every value the metric distance takes), construct a simplicial complex. It can be any complex construction, but make sure it at least includes 0, 1, 2, and 3 dimensional simplices. I have not examined what happens in dimensions higher than 4 dimensional simplices, but feel free to find out and share with all of us. If your code is slow, try to have enough instances through the filtration to be able to find interesting persistent homologies.

5. For each step in the filtration, find the simplicial complex, compute the Euler characteristic, find the homologies, find the ranks of all the homologies, check that the characteristic of the ranks is consistent with the Euler characteristic (a good check to see your code works properly). You need to at least have H0, H1, and H2, but if you can go farther share what you find.

6. Construct some representation of the persistence diagram, and output this. If possible, output it in such a way as can be run through something to graph it or otherwise analyze it, or alternately, graph it in your code.

7. Determine which homologies are persistent and figure out what you can from them. Then provide an analysis of the data.  For example, you may say ‘there seem to be n types of data similar to each other’.  See if you can determine what it means in some sense.